In [2]:
# Instalar Java Tool Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar Spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompactar Spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

# Instalar lib findspark que ajuda a localizar o Spark no sistema e importa-lo como uma biblioteca regular
!pip install -q findspark

In [3]:
# Lib os - Permite Interagir com S.O - Cria variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [4]:
# Inicializa Findspark - Facilita Uso de Spark em Notebook
import findspark
findspark.init()

In [178]:
# Start SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName('Windowed Operation')\
        .master('local')\
        .getOrCreate()

In [179]:
# Define Schema de Dataframe
from pyspark.sql.types import *

esquema = StructType([
            StructField('Pais',   StringType(), True)])

In [180]:
# Cria Dataframe a partir de Fluxo Continuo de Dados
df = spark.readStream.\
             option('sep', ',').\
             option('header', 'false').\
             schema(esquema).\
             csv('Streaming')

In [181]:
# Adicionando uma coluna de timestamp usando a função current_timestamp()
from pyspark.sql.functions import *

df = df.withColumn('timestamp', current_timestamp())

In [182]:
# Consulta Schema do Dataframe
df.printSchema()

root
 |-- Pais: string (nullable = true)
 |-- timestamp: timestamp (nullable = false)



In [183]:
# Gera Dataframe query como uma agregacao de dados do Dataframe rawdata
query = df.select((df.Pais).alias('pais'),(df.timestamp).alias('time')) \
        .groupBy(window('time','30 seconds'),'pais') \
        .count().sort(asc('window'))

In [184]:
# Coleta Dados Objeto de Consulta StreamingQuery query
result = query \
         .writeStream \
         .outputMode('complete') \
         .format('memory') \
         .queryName('DadosFluxo') \
        .start(truncate=False)

In [195]:
# Consulta StreamingQuery em Fluxo de Dados
# Nota: Consulte cada atualização de dados que passar pelo Fluxo
spark.sql("select * from DadosFluxo").show(truncate=False)

+------------------------------------------+----------+-----+
|window                                    |pais      |count|
+------------------------------------------+----------+-----+
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Italia    |2    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Chile     |1    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|USA       |1    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Portugal  |2    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Argentina |1    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Brasil    |2    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Ucrania   |1    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Russia    |1    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|Canada    |1    |
|{2023-11-04 21:33:30, 2023-11-04 21:34:00}|França    |1    |
|{2023-11-04 21:34:00, 2023-11-04 21:34:30}|Russia    |1    |
|{2023-11-04 21:34:00, 2023-11-04 21:34:30}|Brasil    |1    |
|{2023-11-04 21:34:00, 2023-11-04 21:34:30}|USA       |1    |
|{2023-1

In [177]:
# Parar SparkContext - Somente Executar se Precisar Reprocessar o Streaming
spark.stop()